In [ ]:
import os
import openpyxl
import pandas as pd
import numpy as np
from pathlib import Path
# import pyinputplus as pyip
from openpyxl.styles import Alignment, Border, Font, Side

# from caiman_functions.caiman_styles import caiman_excel_styles as ca_ex_st



In [24]:

def imageJ_fluo4_reanalysis_single_file(route):
    file_path = Path(route)
    read_file = pd.read_csv(file_path)
    splitted_file = file_path.stem
    non_space_filename = splitted_file.replace(" ", "_")
    xlsx_append = f"{non_space_filename}.xlsx"
    save_route = os.path.join(file_path.parent, xlsx_append)
    read_file.to_excel(save_route, index=None, header=True)

    wb = openpyxl.load_workbook(save_route)
    sheet = wb.active
    for row in sheet:
        for cell in row:
            # Personal cells style
            if cell.value != None:
                font_title = Font(bold=False)
                cell.font = font_title
                cell.number_format = '0.0000'
                cell.alignment = Alignment(
                    horizontal="center", vertical="center")

    sheet.title = "Time Measurement Report"
    sheet.insert_rows(1)
    time_header = sheet.cell(row=2, column=1).coordinate
    sheet[time_header] = "Time (s)"
    sheet.cell(row=1, column=1).value = splitted_file
    file_max_column = sheet.max_column
    number_cell = 1
    for column in range(1, file_max_column + 1):
        try:
            if "Mean" in sheet.cell(row=2, column=column).value:
                sample_number_header = sheet.cell(
                    row=2, column=column).coordinate
                sheet[sample_number_header] = f"#{number_cell} (Ratio)"
                ca_ex_st.cell_border_style(sample_number_header, sheet)
                number_cell += 1
        except TypeError:
            pass

    # Calculates the widthest value within the cells of a column and set that column width to such a value.
    dims = {}
    for row in sheet:
        for cell in row:
            dims[cell.column_letter] = max(
                (dims.get(cell.column_letter, 0), len(str(cell.value))))
    for col, value in dims.items():
        sheet.column_dimensions[col].width = value

    splitted_file = save_route.split(".xlsx")[0]
    wb.save(save_route)
    print(f"'{file_path.name}' has been converted to '{xlsx_append}'.")


def file_directory_route(file_route):        
        parent = file_route.parent
        stem = file_route.stem
        return parent, stem

In [25]:
file_route = Path(r"H:\20220526_Airyscan\Mam_231_WT_ca_histamina - Z=0_imageJ.csv")
parent, stem = file_directory_route(file_route)

file_path = Path(file_route)
read_file = pd.read_csv(file_path)
df = read_file.rename(columns={" ": "Time(s)"})
basal_fluorescence = df.iloc[:2, 1:].mean()
f_f0 = df.iloc[:, 1:] / basal_fluorescence
df_f_f0 = pd.merge(df.iloc[:,:1], f_f0, left_index=True, right_index=True)

writer = pd.ExcelWriter(os.path.join(parent, f"{stem}_F_F0.xlsx"), engine="xlsxwriter")
workbook = writer.book

# calculates the "analyzed_single_game" DataFrame maximum rows and colums 
max_rows, max_colums = df.shape        
# generates the workbook sheet title
# title = "Normalized_f_f0"
sheet_name = "Fluorescence"
df.to_excel(writer, index=False, sheet_name=sheet_name, startrow=1)
worksheet = writer.sheets[sheet_name]
file_title = f"{stem}"
worksheet.write(0,0, file_title)
# Select the sheet where it is currently writing

sheet_name = "F_F0"
df_f_f0.to_excel(writer, index=False, sheet_name=sheet_name, startrow=1)
worksheet = writer.sheets[sheet_name]
file_title = f"{stem}"
worksheet.write(0,0, file_title)
writer.save()